In [5]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.spatial.distance import cdist
import ipywidgets as widgets
from IPython.display import display

# --- Paramètres de la grille ---
nx, ny = 100, 100
lx, ly = 400, 400
dx, dy = lx / nx, ly / ny
x = np.linspace(0, lx - dx, nx)
y = np.linspace(0, ly - dy, ny)
X, Y = np.meshgrid(x, y)

# --- Modèles de covariance isotropes ---
def spherical_cov(h, range_, sill):
    cov = np.zeros_like(h)
    hr = h / range_
    mask = h <= range_
    cov[mask] = sill * (1 - 1.5 * hr[mask] + 0.5 * hr[mask]**3)
    return cov

def exponential_cov(h, range_, sill):
    return sill * np.exp(-h / range_)

def gaussian_cov(h, range_, sill):
    return sill * np.exp(-(h / range_)**2)

def nugget_cov(h, sill):
    return sill * (h == 0)

# --- FFT-MA 2D périodique ---
def fft_ma_2d(nx, ny, lx, ly, cov_func, range_, sill):
    dx, dy = lx / nx, ly / ny
    x = np.linspace(0, lx - dx, nx)
    y = np.linspace(0, ly - dy, ny)
    X, Y = np.meshgrid(x, y)

    def periodic_distance(X, Y):
        Xc = np.minimum(X, lx - X)
        Yc = np.minimum(Y, ly - Y)
        return np.sqrt(Xc**2 + Yc**2)

    dist = periodic_distance(np.abs(X), np.abs(Y))
    C = cov_func(dist, range_, sill)
    S = np.fft.fft2(C)
    S = np.maximum(S, 0)
    Z = np.random.normal(size=(ny, nx)) + 1j * np.random.normal(size=(ny, nx))
    Z /= np.sqrt(2)
    F = Z * np.sqrt(S)
    field = np.fft.ifft2(F).real * np.sqrt(nx * ny)
    return field

# --- Krigeage simple 2D ---
def simple_kriging_2d(grid_points, sample_points, sample_values, cov_func, mean=0):
    d_grid_samples = cdist(grid_points, sample_points)
    d_samples = cdist(sample_points, sample_points)
    C = cov_func(d_samples)
    c0 = cov_func(d_grid_samples)
    weights = np.linalg.solve(C, c0.T)
    y_est = mean + weights.T @ (sample_values - mean)
    sigma2 = cov_func(np.zeros(1))[0] - np.sum(weights * c0.T, axis=0)
    return y_est, sigma2, weights

# --- Krigeage ordinaire 2D ---
def ordinary_kriging_2d(grid_points, sample_points, sample_values, cov_func):
    d_grid_samples = cdist(grid_points, sample_points)
    d_samples = cdist(sample_points, sample_points)
    n = len(sample_points)
    C = cov_func(d_samples)
    C_aug = np.zeros((n + 1, n + 1))
    C_aug[:n, :n] = C
    C_aug[n, :n] = 1
    C_aug[:n, n] = 1
    y_est, sigma2, weights_list = [], [], []
    for i in range(len(grid_points)):
        c0 = cov_func(d_grid_samples[i])
        c_aug = np.append(c0, 1)
        sol = np.linalg.solve(C_aug, c_aug)
        weights, lam = sol[:-1], sol[-1]
        y_est.append(weights @ sample_values)
        sigma2.append(cov_func(np.zeros(1))[0] - weights @ c0 - lam)
        weights_list.append(weights)
    return np.array(y_est), np.array(sigma2), np.array(weights_list)

# --- Widgets ---
model_options = ['Sphérique', 'Exponentiel', 'Gaussien', 'Pépite']

def model_to_func(name):
    return {
        'Sphérique': spherical_cov,
        'Exponentiel': exponential_cov,
        'Gaussien': gaussian_cov,
        'Pépite': lambda h, r, s: nugget_cov(h, s)
    }[name]

sill_w = widgets.FloatSlider(value=1.0, min=0, max=5, step=0.1, description='Sill')
range_w = widgets.FloatSlider(value=20, min=1, max=100, step=1, description='Portée')
model_w = widgets.Dropdown(options=model_options, value='Sphérique', description='Modèle')
mean_w = widgets.FloatSlider(value=0, min=-5, max=5, step=0.1, description='Moyenne champ')

# 10 000 points d’échantillon aléatoires uniformes
np.random.seed(42)
sample_points = np.column_stack((
    np.random.uniform(0, lx, 10000),
    np.random.uniform(0, ly, 10000)
))

grid_points = np.column_stack([X.ravel(), Y.ravel()])

def update(sill, range_, model, mean):
    cov_func = model_to_func(model)
    field = fft_ma_2d(nx, ny, lx, ly, cov_func, range_, sill) + mean

    # Trouver valeurs aux points échantillons
    sample_values = []
    for sp in sample_points:
        ix = int(sp[0] / dx)
        iy = int(sp[1] / dy)
        sample_values.append(field[iy, ix])
    sample_values = np.array(sample_values)

    y_ks, var_ks, _ = simple_kriging_2d(grid_points, sample_points, sample_values, lambda h: cov_func(h, range_, sill), mean)
    y_ko, var_ko, _ = ordinary_kriging_2d(grid_points, sample_points, sample_values, lambda h: cov_func(h, range_, sill))

    field_img = field.reshape(ny, nx)
    y_ks_img = y_ks.reshape(ny, nx)
    var_ks_img = var_ks.reshape(ny, nx)
    y_ko_img = y_ko.reshape(ny, nx)
    var_ko_img = var_ko.reshape(ny, nx)

    fig, axs = plt.subplots(2, 3, figsize=(18, 10))

    im0 = axs[0,0].imshow(field_img, origin='lower', extent=[0,lx,0,ly])
    axs[0,0].set_title('Champ simulé (FFT-MA périodique)')
    fig.colorbar(im0, ax=axs[0,0])

    axs[0,1].axis('off')

    im2 = axs[0,2].imshow(y_ks_img, origin='lower', extent=[0,lx,0,ly])
    axs[0,2].set_title('Estimation Krigeage simple')
    fig.colorbar(im2, ax=axs[0,2])

    im3 = axs[1,0].imshow(var_ks_img, origin='lower', extent=[0,lx,0,ly])
    axs[1,0].set_title('Variance estimation KS')
    fig.colorbar(im3, ax=axs[1,0])

    im4 = axs[1,1].imshow(y_ko_img, origin='lower', extent=[0,lx,0,ly])
    axs[1,1].set_title('Estimation Krigeage ordinaire')
    fig.colorbar(im4, ax=axs[1,1])

    im5 = axs[1,2].imshow(var_ko_img, origin='lower', extent=[0,lx,0,ly])
    axs[1,2].set_title('Variance estimation KO')
    fig.colorbar(im5, ax=axs[1,2])

    for ax in axs.flat:
        ax.set_xlabel('X')
        ax.set_ylabel('Y')

    plt.tight_layout()
    plt.show()

widgets.interact(update, sill=sill_w, range_=range_w, model=model_w, mean=mean_w)


interactive(children=(FloatSlider(value=1.0, description='Sill', max=5.0), FloatSlider(value=20.0, description…

<function __main__.update(sill, range_, model, mean)>